In [36]:
from google.colab import drive
drive.mount('/content/drive')
from datetime import datetime
from keras.callbacks import History 
history = History()
from jinja2 import Environment, FileSystemLoader
import json
import numpy as np
import os
import pandas as pd
import sys 
import tensorflow as tf
import matplotlib.pyplot as plt
import math

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Parameters to specify

In [37]:
#example: Q16.8 -> precision = 8, wordsize = 24, double = 48
#specify:
PRECISION = 24
INTEGER = 24
WORDSIZE = INTEGER + PRECISION
D_WORDSIZE = 2*WORDSIZE
SLACK = 8-(WORDSIZE%8)



nn_topo_title = '/content/drive/MyDrive/NN-P4/nn-reports/nn_model_parameters.json'
load_model_title = f'/content/drive/MyDrive/NN-P4/tf-models/model_with_24_attributes_24_hidden_nodes.csv'
path_test = f'/content/drive/MyDrive/NN-P4/csv-files/df_6_test_24_attr_10000_samples.csv'
path_test_target = f'/content/drive/MyDrive/NN-P4/csv-files/df_6_test_target_10000_samples.csv'


In [38]:
n_bits = WORDSIZE
#for jinja template
wordsize = WORDSIZE
precision = PRECISION
slack = SLACK
print(slack)

8


### Load NN parameters and model

In [39]:
#Load file with NN topology
with open(nn_topo_title, "r") as f:
  rep = json.load(f)

#Get NN topology and information to load correct dataframes
attributes_array = rep["attributes"]
number_of_attributes = rep["number_of_attributes"]
hidden_layer_nodes = rep["hidden_layer_nodes"]
output_layer_nodes = rep["output_layer_nodes"]
accuracy_test = rep["accuracy_test"]
batch_size = rep["batch_size"]
epochs = rep["epochs"]
df_number = 6
# df_number = rep["df_number"]

#load dataframes and adequate them for file generation
df_test_raw = pd.read_csv(path_test)
display(df_test_raw)
df_test = df_test_raw[attributes_array]
display(df_test)
target_test = pd.read_csv(path_test_target)


#load model to get weights
print(load_model_title)
model = tf.keras.models.load_model(load_model_title)

norm_layer_weights = model.layers[0].get_weights()[0] #weight
norm_layer_biases  = model.layers[0].get_weights()[1] #bias
hidden_layer_weights = model.layers[1].get_weights()[0] #weight
hidden_layer_biases  = model.layers[1].get_weights()[1] #bias
out_layer_weights = model.layers[2].get_weights()[0] #weight
out_layer_biases  = model.layers[2].get_weights()[1] #bias
print('\nInput -> Normalization Layer bias:\n',norm_layer_biases)
print('\nInput -> Normalization Layer weights:\n',norm_layer_weights)
print('\nNormalization -> Hidden Layer bias:\n',hidden_layer_biases)
print('\nNormalization -> Hidden Layer weights:\n',hidden_layer_weights)
print('\nHidden -> Output Layer bias:\n',out_layer_biases)
# print('\nHidden -> Output Layer weights:\n',out_layer_weights,'\n')

,proto,f_pktTotalCount,f_octetTotalCount,f_min_ps,f_max_ps,f_avg_ps,f_flowStart,f_flowEnd,f_flowDuration,f_min_piat,...,b_min_ps,b_max_ps,b_avg_ps,b_flowStart,b_flowEnd,b_flowDuration,b_min_piat,b_max_piat,b_avg_piat,flowEndReason
0,17,10,563,48,131,56,1555964273,1555964836358,563,3,...,0,0,0,0,0,0,0,0,0,2
1,6,10,6167,40,2960,616,1556053574,1556053636477,61,0,...,40,4348,1056,1556053574,1556053636,1554497582902,0,60,4,3
2,6,4,367,40,235,91,1555955309,1555955309156,0,0,...,40,1616,569,1555955309,1555955309,1554399353845,0,0,0,2
3,6,14,2935,40,824,209,1556047747,1556047748715,1,0,...,40,2285,280,1556047747,1556047748,1554491700967,0,0,0,3
4,6,16,3417,40,906,213,1556029424,1556029664331,240,0,...,40,464,106,1556029424,1556029664,1554473634906,0,239,13,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,6,1,40,40,40,40,1556124962,1556124962,0,0,...,0,0,0,0,0,0,0,0,0,2
9996,6,7,1014,40,557,144,1556054273,1556054274031,0,0,...,40,2876,714,1556054273,1556054273,1554498219247,0,0,0,4
9997,6,70,13992,40,1570,199,1555969058,1555969315453,256,0,...,40,1509,387,1555969058,1555969315,1554413346394,0,45,2,3
9998,6,31,2674,52,588,86,1555967356,1555967641164,285,0,...,52,2888,1267,1555967356,1555967641,1554411673807,0,239,10,3


,proto,f_pktTotalCount,f_octetTotalCount,f_min_ps,f_max_ps,f_avg_ps,f_flowStart,f_flowEnd,f_flowDuration,f_min_piat,...,b_min_ps,b_max_ps,b_avg_ps,b_flowStart,b_flowEnd,b_flowDuration,b_min_piat,b_max_piat,b_avg_piat,flowEndReason
0,17,10,563,48,131,56,1555964273,1555964836358,563,3,...,0,0,0,0,0,0,0,0,0,2
1,6,10,6167,40,2960,616,1556053574,1556053636477,61,0,...,40,4348,1056,1556053574,1556053636,1554497582902,0,60,4,3
2,6,4,367,40,235,91,1555955309,1555955309156,0,0,...,40,1616,569,1555955309,1555955309,1554399353845,0,0,0,2
3,6,14,2935,40,824,209,1556047747,1556047748715,1,0,...,40,2285,280,1556047747,1556047748,1554491700967,0,0,0,3
4,6,16,3417,40,906,213,1556029424,1556029664331,240,0,...,40,464,106,1556029424,1556029664,1554473634906,0,239,13,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,6,1,40,40,40,40,1556124962,1556124962,0,0,...,0,0,0,0,0,0,0,0,0,2
9996,6,7,1014,40,557,144,1556054273,1556054274031,0,0,...,40,2876,714,1556054273,1556054273,1554498219247,0,0,0,4
9997,6,70,13992,40,1570,199,1555969058,1555969315453,256,0,...,40,1509,387,1555969058,1555969315,1554413346394,0,45,2,3
9998,6,31,2674,52,588,86,1555967356,1555967641164,285,0,...,52,2888,1267,1555967356,1555967641,1554411673807,0,239,10,3


/content/drive/MyDrive/NN-P4/tf-models/model_with_24_attributes_24_hidden_nodes.csv

Input -> Normalization Layer bias:
 [2.9698288e+01 2.8311112e+06 5.6234806e+12 4.3124434e+03 6.7711431e+05
 4.5378461e+04 3.5404536e+09 6.0041154e+23 3.2961770e+04 1.8467079e+03
 5.9219204e+03 2.1278418e+03 7.8261220e+06 3.4470118e+13 4.7658374e+03
 9.1206990e+06 4.0341178e+05 2.2189753e+17 2.2189754e+17 6.0302853e+23
 1.0987889e+03 3.3089800e+03 1.2074968e+03 7.9765946e-01]

Input -> Normalization Layer weights:
 [1.0633804e+01 4.0723248e+01 1.8743688e+04 5.8449261e+01 4.3509113e+02
 1.3653365e+02 1.5560476e+09 8.3975517e+11 5.1716328e+01 2.9954376e+00
 2.0367210e+01 6.0951538e+00 6.0243073e+01 1.0529434e+05 6.9839951e+01
 1.2236459e+03 3.0791229e+02 1.3972297e+09 1.3972297e+09 7.4549658e+11
 1.7568840e+00 1.5753778e+01 3.9566040e+00 2.6852784e+00]

Normalization -> Hidden Layer bias:
 [-0.23383279 -0.7049143  -1.7622015  -2.3540123  -1.0679823   0.03289792
 -1.1902038  -2.766884    0.35906738 -1.0855

In [40]:
def num_to_c2(num, wordsize):
  if(num < 0):
    return 2**wordsize + num
  return num

### file generator for P4 csv input

In [41]:
df_test.to_csv(f"/content/drive/MyDrive/network-traffic-classification-main/code/csv_files_input/df_test_6_with_24_attributes.csv", index=False)
target_test.to_csv(f"/content/drive/MyDrive/network-traffic-classification-main/csv-files/test_target_6_with_24_attributes.csv", index=False)
# df_test.to_csv(f"/content/drive/MyDrive/network-traffic-classification-main/code/csv_files_input/df_test_{df_number}_with_{number_of_attributes}_attributes.csv", index=False)
# target_test.to_csv(f"/content/drive/MyDrive/network-traffic-classification-main/csv-files/test_target_{df_number}_with_{number_of_attributes}_attributes.csv", index=False)

###  file generator for TF predictions 
to be compared to P4_prediction in the analiser

In [42]:
tf_predictions_probabilities = model.predict(df_test)
tf_loss, tf_acc =  model.evaluate(df_test,  target_test, verbose=2, batch_size=batch_size)

tf_predictions = []

for i,x in enumerate(tf_predictions_probabilities):
  #print("i:",i,"x_max:",x.max(),"x:",x)  
  j_max = x.argmax()
  tf_predictions.append(j_max)  

conf_m = tf.math.confusion_matrix(target_test,tf_predictions)
print(conf_m)

df_tf = pd.DataFrame(tf_predictions_probabilities)
df_tf.insert(loc = 0, column = "tf_predictions", value = tf_predictions)
#print(df_tf)

title_for_analiser = "/content/drive/MyDrive/network-traffic-classification-main/csv-files/tf_outputs_classifications.csv"
df_tf.to_csv(title_for_analiser, index=False)  


313/313 [==============================] - 1s 2ms/step
1000/1000 - 1s - loss: 0.8247 - accuracy: 0.7413 - 1s/epoch - 1ms/step
tf.Tensor(
[[5032  432   93   59    2   14    4    1    1    0    0    3   12    0
     0    0    0    1    0    0    0    0    0]
 [ 460 1130    4    0    0    0    0    0    3    0    4    7    0    0
     0    0    0    0    0    0    0    0    0]
 [  36    3  646    6    0    3    1    2    0    0    0    5    0    0
     0    0    0    0    0    0    0    0    0]
 [ 236   47   11  155    0   42    0    0    0    0    0    0    1    0
     0    0    0    0    0    0    0    0    0]
 [ 174   47    0    0   43    1    0    0    0    0    1    0    0    0
     0    0    0    0    0    0    0    0    0]
 [ 132   22    3   31    0   74    1    5    0    0    1    0    2    0
     0    0    0    0    0    0    0    0    0]
 [ 146   49    0    0    3    0    0    0    0    0    0    1    0    0
     0    0    0    0    0    0    0    0    0]
 [  40   14    0    0  

### Topology file generator for sx

In [43]:
weights = norm_layer_weights
biases = norm_layer_biases
#number of norm_layer_nodes = number_of_attributes
for i in range(number_of_attributes):
  sx_id = 1+i    
  sx_entries = {
    "target": "bmv2",
    "p4info": "build/ANN.p4.p4info.txt",
    "bmv2_json": "build/ANN.json",
    "table_entries": [
      {
        "table": "MyIngress.ann_forward",
        "match": {
          "standard_metadata.ingress_port": 1
        },
        "action_name": "MyIngress.mcast",
        "action_params": {
          "mgroup": sx_id
        }
      },
      {
        "table": "MyIngress.tab_neuron_id",
        "default_action": True,
        "action_name": "MyIngress.set_neuron_id",
        "action_params": {
          "neuron_id": sx_id
        }
      },
      {
        "table": "MyIngress.tab_n_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_n_expected_stimuli",
        "action_params": {
          "n_expected_stimuli": 1
        }
      },
      {
        "table": "MyIngress.tab_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_expected_stimuli",
        "action_params": {
          "expected_stimuli": 1
        }
      },
      {
        "table": "MyIngress.tab_agg_func",
        "default_action": True,
        "action_name": "MyIngress.set_agg_func",
        "action_params": {
          "agg_func": 5
        }
      },
      {
        "table": "MyIngress.tab_activation_func",
        "default_action": True,
        "action_name": "MyIngress.set_activation_func",
        "action_params": {
          "activation_func": 2
        }
      }
    ],
    "multicast_group_entries":[
      {
        "multicast_group_id" : sx_id,
        "replicas": []
      }
    ]
  }  
  for h in range(hidden_layer_nodes):
    sx_entries["multicast_group_entries"][0]["replicas"].append(
      {
        "egress_port" :51+h,
        "instance": 1
      }
    ) 



  #print(sx_entries["table_entries"])
  attr_name = attributes_array[i]
  
  for j in sorted(df_test[attr_name].unique()):
    # if attr_name == "83":  
    #   #print(j)
    #   if j == 0 or j == 1460:

    j = int(j)
    print(f"j:{j} ,weights[i]:{weights[i]}, math.sqrt(biases[i]:{math.sqrt(biases[i])}")

    normalized_value = (j - weights[i]) / math.sqrt(biases[i])
    print(f"normalized_value:{normalized_value}")
  
    fp_normalized_value = int(normalized_value * (2**PRECISION))  
    print(f"fp_normalized_value:{fp_normalized_value}")

    c2_normalized_value = num_to_c2(fp_normalized_value, n_bits)
    print(f"c2_normalized_value:{c2_normalized_value}")

    sx_entries["table_entries"].append(
      {
        "table": "MyIngress.tab_normalization",
        "match": {
          "hdr.ann.data": j
        },
        "action_name": "MyIngress.set_normalization",
        "action_params": {
          "normalized_data": c2_normalized_value
        }
      }
    )


  print(sx_entries["table_entries"])

  sx_file_title = "/content/drive/MyDrive/network-traffic-classification-main/code/topology/s%d"%sx_id + "-runtime.json"
  with open(sx_file_title, "w") as f:
    json.dump(sx_entries, f, indent = 2)

print(json.dumps(sx_entries, indent = 2))

A saída de streaming foi truncada nas últimas 5000 linhas.
normalized_value:1.0418798815449246
fp_normalized_value:17479843
c2_normalized_value:17479843
j:1554567480635 ,weights[i]:745496576000.0, math.sqrt(biases[i]:776549117146.1053
normalized_value:1.0418798846986208
fp_normalized_value:17479843
c2_normalized_value:17479843
j:1554567485786 ,weights[i]:745496576000.0, math.sqrt(biases[i]:776549117146.1053
normalized_value:1.0418798913318135
fp_normalized_value:17479843
c2_normalized_value:17479843
j:1554567491051 ,weights[i]:745496576000.0, math.sqrt(biases[i]:776549117146.1053
normalized_value:1.0418798981118096
fp_normalized_value:17479844
c2_normalized_value:17479844
j:1554567492543 ,weights[i]:745496576000.0, math.sqrt(biases[i]:776549117146.1053
normalized_value:1.0418799000331305
fp_normalized_value:17479844
c2_normalized_value:17479844
j:1554567517213 ,weights[i]:745496576000.0, math.sqrt(biases[i]:776549117146.1053
normalized_value:1.0418799318018872
fp_normalized_value:17479

### Topology file generator for s5x

In [44]:
weights = hidden_layer_weights
biases = hidden_layer_biases
expected_stimuli = sum([1<<(i + 1) for i in range(number_of_attributes)])

for i in range(hidden_layer_nodes):
  s5x_id = 51+i    
  s5x_entries = {
    "target": "bmv2",
    "p4info": "build/ANN.p4.p4info.txt",
    "bmv2_json": "build/ANN.json",
    "table_entries": [
      {
        "table": "MyIngress.tab_neuron_id",
        "default_action": True,
        "action_name": "MyIngress.set_neuron_id",
        "action_params": {
          "neuron_id": s5x_id
        }
      },
      {
        "table": "MyIngress.tab_n_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_n_expected_stimuli",
        "action_params": {
          "n_expected_stimuli": number_of_attributes
        }
      },
      {
        "table": "MyIngress.tab_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_expected_stimuli",
        "action_params": {
          "expected_stimuli": expected_stimuli
        }
      },
      {
        "table": "MyIngress.tab_agg_func",
        "default_action": True,
        "action_name": "MyIngress.set_agg_func",
        "action_params": {
          "agg_func": 1
        }
      },
      {
        "table": "MyIngress.tab_activation_func",
        "default_action": True,
        "action_name": "MyIngress.set_activation_func",
        "action_params": {
          "activation_func": 3
        }
      }
    ],
    "multicast_group_entries":[
      {
        "multicast_group_id" : s5x_id,
        "replicas":[]
      }
    ]
  } 

  for h in range(output_layer_nodes):
    s5x_entries["multicast_group_entries"][0]["replicas"].append(
      {
        "egress_port" :101+h,
        "instance": 1
      }
    ) 

  fp_value = int(biases[i] * (2**PRECISION))  
  c2_value = num_to_c2(fp_value, n_bits)  
  s5x_entries["table_entries"].append(
    {
      "table": "MyIngress.tab_neuron_bias",
      "default_action": True,
      "action_name": "MyIngress.set_neuron_bias",
      "action_params": {
        "neuron_bias": c2_value
      }
    },
  )
  for j in range(number_of_attributes):
    fp_value = int(weights[j][i] * (2**PRECISION))  
    c2_value = num_to_c2(fp_value, n_bits)  
    s5x_entries["table_entries"].append(
      {
        "table": "MyIngress.tab_n2n_weight",
        "match": {
          "hdr.ann.neuron_id": j+1
        },
        "action_name": "MyIngress.set_n2n_weight",
        "action_params": {
          "n2n_weight":  c2_value
        }
      },
    )
    s5x_entries["table_entries"].append(
      {
        "table": "MyIngress.ann_forward",
        "match": {
          "standard_metadata.ingress_port": j+1
        },
        "action_name": "MyIngress.mcast",
        "action_params": {
          "mgroup": s5x_id
        }
      }
    )
  s5x_file_title = "/content/drive/MyDrive/network-traffic-classification-main/code/topology/s%d"%s5x_id + "-runtime.json"
  with open(s5x_file_title, "w") as f:
    json.dump(s5x_entries, f, indent = 2)

print(json.dumps(s5x_entries, indent = 2))

{
  "target": "bmv2",
  "p4info": "build/ANN.p4.p4info.txt",
  "bmv2_json": "build/ANN.json",
  "table_entries": [
    {
      "table": "MyIngress.tab_neuron_id",
      "default_action": true,
      "action_name": "MyIngress.set_neuron_id",
      "action_params": {
        "neuron_id": 62
      }
    },
    {
      "table": "MyIngress.tab_n_expected_stimuli",
      "default_action": true,
      "action_name": "MyIngress.set_n_expected_stimuli",
      "action_params": {
        "n_expected_stimuli": 24
      }
    },
    {
      "table": "MyIngress.tab_expected_stimuli",
      "default_action": true,
      "action_name": "MyIngress.set_expected_stimuli",
      "action_params": {
        "expected_stimuli": 33554430
      }
    },
    {
      "table": "MyIngress.tab_agg_func",
      "default_action": true,
      "action_name": "MyIngress.set_agg_func",
      "action_params": {
        "agg_func": 1
      }
    },
    {
      "table": "MyIngress.tab_activation_func",
      "default_action

In [45]:
# #expected stimuli
# #hidden
# sum([1<<(i + 1) for i in range(n)])

# #output
# sum([1<<(i + 51) for i in range(n)])

# SWITCH S126:
# expected_stimuli = sum([1<<(i + 1) for i in range(24)])
# print(expected_stimuli)

# #expected stimuli 
# import sys

# def main(neurons):
#     neurons = [int(x) for x in neurons.split(",")]
#     expected_stimuli = 0
#     for x in neurons:
#         expected_stimuli = expected_stimuli | 1 << x

#     print("dec:", expected_stimuli)
#     print("bin:", bin(expected_stimuli))

# if __name__ == '__main__':
#     main(sys.argv[1])

# #6
# #15762598695796736

33554430


### Topology file generator for s1xx

In [46]:
weights = out_layer_weights
biases = out_layer_biases
expected_stimuli = sum([1<<(i + 51) for i in range(hidden_layer_nodes)])

for i in range(output_layer_nodes):
  s1xx_id = 101+i    
  s1xx_entries = {
    "target": "bmv2",
    "p4info": "build/ANN.p4.p4info.txt",
    "bmv2_json": "build/ANN.json",
    "table_entries": [
      {
        "table": "MyIngress.tab_neuron_id",
        "default_action": True,
        "action_name": "MyIngress.set_neuron_id",
        "action_params": {
          "neuron_id": s1xx_id
        }
      },
      {
        "table": "MyIngress.tab_n_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_n_expected_stimuli",
        "action_params": {
          "n_expected_stimuli": hidden_layer_nodes
        }
      },
      {
        "table": "MyIngress.tab_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_expected_stimuli",
        "action_params": {
          "expected_stimuli": expected_stimuli
          
        }
      },
      {
        "table": "MyIngress.tab_agg_func",
        "default_action": True,
        "action_name": "MyIngress.set_agg_func",
        "action_params": {
          "agg_func": 1
        }
      },
      {
        "table": "MyIngress.tab_activation_func",
        "default_action": True,
        "action_name": "MyIngress.set_activation_func",
        "action_params": {
          "activation_func": 2
        }
      }
    ],
    "multicast_group_entries":[
      {
        "multicast_group_id" : s1xx_id,
        "replicas":[
          {
            "egress_port" :126,
            "instance": 1
          }
        ]
      }
    ]
  }

  fp_value = int(biases[i] * (2**PRECISION))  
  c2_value = num_to_c2(fp_value, n_bits)  
  s1xx_entries["table_entries"].append(
    {
      "table": "MyIngress.tab_neuron_bias",
      "default_action": True,
      "action_name": "MyIngress.set_neuron_bias",
      "action_params": {
        "neuron_bias": c2_value
      }
    },
  )

  for j in range(hidden_layer_nodes):
    fp_value = int(weights[j][i] * (2**PRECISION))  
    c2_value = num_to_c2(fp_value, n_bits)  
    s1xx_entries["table_entries"].append(
      {
        "table": "MyIngress.tab_n2n_weight",
        "match": {
          "hdr.ann.neuron_id": j+51
        },
        "action_name": "MyIngress.set_n2n_weight",
        "action_params": {
          "n2n_weight":  c2_value
        }
      }
    )
    s1xx_entries["table_entries"].append(
      {
        "table": "MyIngress.ann_forward",
        "match": {
          "standard_metadata.ingress_port": j+51
        },
        "action_name": "MyIngress.mcast",
        "action_params": {
          "mgroup": s1xx_id
        }
      }
    )

  #print(s1xx_entries["table_entries"])
  s1xx_file_title = "/content/drive/MyDrive/network-traffic-classification-main/code/topology/s%d"%s1xx_id + "-runtime.json"
  with open(s1xx_file_title, "w") as f:
    json.dump(s1xx_entries, f, indent = 2)

print(json.dumps(s1xx_entries, indent = 2))

{
  "target": "bmv2",
  "p4info": "build/ANN.p4.p4info.txt",
  "bmv2_json": "build/ANN.json",
  "table_entries": [
    {
      "table": "MyIngress.tab_neuron_id",
      "default_action": true,
      "action_name": "MyIngress.set_neuron_id",
      "action_params": {
        "neuron_id": 124
      }
    },
    {
      "table": "MyIngress.tab_n_expected_stimuli",
      "default_action": true,
      "action_name": "MyIngress.set_n_expected_stimuli",
      "action_params": {
        "n_expected_stimuli": 12
      }
    },
    {
      "table": "MyIngress.tab_expected_stimuli",
      "default_action": true,
      "action_name": "MyIngress.set_expected_stimuli",
      "action_params": {
        "expected_stimuli": 9221120237041090560
      }
    },
    {
      "table": "MyIngress.tab_agg_func",
      "default_action": true,
      "action_name": "MyIngress.set_agg_func",
      "action_params": {
        "agg_func": 1
      }
    },
    {
      "table": "MyIngress.tab_activation_func",
      "de

### Topology file generator for topology.json


In [47]:
#definition of topology
topology = {  
    'hosts': {
        'h1': {'ip': '10.0.1.1/24', 'mac': '08:00:00:00:01:11',
                'commands':['route add default gw 10.0.1.10 dev eth0',
                            'arp -i eth0 -s 10.0.1.10 08:00:00:00:01:00']},
        'h2': {'ip': '10.0.2.2/24', 'mac': '08:00:00:00:02:22',
                'commands':['route add default gw 10.0.2.20 dev eth0',
                            'arp -i eth0 -s 10.0.2.20 08:00:00:00:02:00']}
    },
    'switches': {},
    'links': []
}

#populate switches dict
for i in range(number_of_attributes):
  switch_name = 's%d'%(i+1)
  switch_content = { 'runtime_json' : 'topology/s%d'%(i+1) +'-runtime.json' }
  topology['switches'][switch_name] = switch_content

for i in range(hidden_layer_nodes):
  switch_name = 's%d'%(i+51)
  switch_content = { 'runtime_json' : 'topology/s%d'%(i+51) +'-runtime.json' }
  topology['switches'][switch_name] = switch_content

for i in range(output_layer_nodes):
  switch_name = 's%d'%(i+101)
  switch_content = { 'runtime_json' : 'topology/s%d'%(i+101) +'-runtime.json' }
  topology['switches'][switch_name] = switch_content

topology['switches']['s126'] = { 'runtime_json' : 'topology/s126-runtime.json' }

#populate links list
for i in range(number_of_attributes):
  topology['links'].append(["h1", "s%d"%(i+1)+"-p1"])

for i in range(number_of_attributes):
  for j in range(hidden_layer_nodes): 
    topology['links'].append(["s%d"%(i+1)+"-p%d"%(j+51), "s%d"%(j+51)+"-p%d"%(i+1)])  

for i in range(hidden_layer_nodes):
  for j in range(output_layer_nodes): 
    topology['links'].append(["s%d"%(i+51)+"-p%d"%(j+101), "s%d"%(j+101)+"-p%d"%(i+51)])  

for i in range(output_layer_nodes):    
  topology['links'].append(["s%d"%(i+101)+"-p126", "s126-p%d"%(i+101)])  

topology['links'].append(["s126-p2", "h2"])

display(topology)

topology_title = "/content/drive/MyDrive/network-traffic-classification-main/code/topology/topology.json"
with open(topology_title, "w") as f:
  json.dump(topology, f, indent = 2)


{'hosts': {'h1': {'ip': '10.0.1.1/24',
   'mac': '08:00:00:00:01:11',
   'commands': ['route add default gw 10.0.1.10 dev eth0',
    'arp -i eth0 -s 10.0.1.10 08:00:00:00:01:00']},
  'h2': {'ip': '10.0.2.2/24',
   'mac': '08:00:00:00:02:22',
   'commands': ['route add default gw 10.0.2.20 dev eth0',
    'arp -i eth0 -s 10.0.2.20 08:00:00:00:02:00']}},
 'switches': {'s1': {'runtime_json': 'topology/s1-runtime.json'},
  's2': {'runtime_json': 'topology/s2-runtime.json'},
  's3': {'runtime_json': 'topology/s3-runtime.json'},
  's4': {'runtime_json': 'topology/s4-runtime.json'},
  's5': {'runtime_json': 'topology/s5-runtime.json'},
  's6': {'runtime_json': 'topology/s6-runtime.json'},
  's7': {'runtime_json': 'topology/s7-runtime.json'},
  's8': {'runtime_json': 'topology/s8-runtime.json'},
  's9': {'runtime_json': 'topology/s9-runtime.json'},
  's10': {'runtime_json': 'topology/s10-runtime.json'},
  's11': {'runtime_json': 'topology/s11-runtime.json'},
  's12': {'runtime_json': 'topology/

### Jinja template to generate ann.p4, ann-header.py and ann_tester.py

In [48]:
environment = Environment(loader=FileSystemLoader("/content/drive/MyDrive/network-traffic-classification-main/jinja/templates/"))
template = environment.get_template("ANN_tester_b.py")
content = template.render(
    wordsize= wordsize,
    precision= precision,
    integer= wordsize - precision,
    fraction = precision,
    slack = slack,
    df_number = 6,
    number_of_attributes = number_of_attributes

)
filename = "/content/drive/MyDrive/network-traffic-classification-main/code/ANN_tester_b.py"
with open(filename, mode="w", encoding="utf-8") as message:
    message.write(content)
    print(f"... wrote {filename}")

###################################################  
template = environment.get_template("ANN_hdrs.py")
content = template.render(
    wordsize= wordsize,
    slack = slack,
)
filename = "/content/drive/MyDrive/network-traffic-classification-main/code/ANN_hdrs.py"
with open(filename, mode="w", encoding="utf-8") as message:
    message.write(content)
    print(f"... wrote {filename}")

################################################### 
template = environment.get_template("ANN.p4")
content = template.render(
    wordsize= wordsize,
    precision= precision,
    double = wordsize*2,
    slack = slack,
)
filename = "/content/drive/MyDrive/network-traffic-classification-main/code/ANN.p4"
with open(filename, mode="w", encoding="utf-8") as message:
    message.write(content)
    print(f"... wrote {filename}")

... wrote /content/drive/MyDrive/network-traffic-classification-main/code/ANN_tester_b.py
... wrote /content/drive/MyDrive/network-traffic-classification-main/code/ANN_hdrs.py
... wrote /content/drive/MyDrive/network-traffic-classification-main/code/ANN.p4
